In [16]:
import os
import getpass

import pandas as pd
import matplotlib.pyplot as plt

from typing import Sequence

import autogen
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMessageTermination, TimeoutTermination, MaxMessageTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage, MultiModalMessage
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console

In [ ]:
# Load your API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
# Create OpenAI model client for inference
openai_model_client = OpenAIChatCompletionClient(
    model = "gpt-4o-mini",
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [ ]:
# Successful basic user test!

# test_user_message = UserMessage(content = "What is the capital of France?", source = "user")
# result = await openai_model_client.create([test_user_message])
# print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=8) cached=False logprobs=None thought=None


In [17]:
# Define all model tools
def search_web_tool(query: str) -> str:
    """Searches web for relevant information."""
    # Call API!
    return "The start was 10 and the end was 15."

def percentage_change_tool(start: float, end: float) -> float:
    """Finds the percentage change between start and end."""
    return ((end - start) / start) * 100

def addition_tool(first: float, second: float) -> float:
    """Adds two numbers together."""
    return first + second

In [18]:
# Define all agents
planning_agent = AssistantAgent(
    name = "PlanningAgent",
    model_client = openai_model_client,
    description = "An agent for planning tasks, this agent should be the first to engage the task.",
    system_message = """
        You are a planning agent.
        Your job is to break down complex tasks into smaller, manageable subtasks.
        Your team members are:
            - web_search_agent: Searches for information
            - data_manipulator_agent: Performs calculations and ranks results by relevance

        You only plan and delegate tasks - you do not execute them yourself.

        When assigning tasks, use this format:
        N. <agent> : <task>

        After all tasks are complete, summarize the findings and end with "TERMINATE".
    """
)

web_search_agent = AssistantAgent(
    name = "WebSearchAgent",
    model_client = openai_model_client,
    description = "A web search agent for relevant results.",
    tools = [search_web_tool],
    system_message = """
        You are a web search agent.
        Your only tool is search_web_tool - use it to find information.
        You make only one search call at a time.
        Once you have the results, you never do calculations based on them.
    """
)

data_manipulator_agent = AssistantAgent(
    name = "DataManipulatorAgent",
    model_client = openai_model_client,
    description = "A data manipulator agent. Useful for performing calculations and ranking items according to relevance.",
    tools = [percentage_change_tool, addition_tool],
    system_message = """
        You are a data manipulator agent.
        Given the tasks you have been assigned, you should analyze the data and provide results.
    """
)

In [21]:
# Set up the multi-agent team, rotation, and termination conditions

text_mention_termination = TextMessageTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages = 25)
timeout_termination = TimeoutTermination(timeout_seconds = 10)

termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    participants = [planning_agent, web_search_agent, data_manipulator_agent],
    model_client = openai_model_client,
    termination_condition = termination
)

In [ ]:
# Run a task through the multi-agent system

task = "What is the percentage change?"

await Console(team.run_stream(task = task))

---------- user ----------
What is the percentage change?
---------- PlanningAgent ----------
To provide you with a detailed response on percentage change, I first need to clarify your request. Percentage change usually involves comparing an initial value to a final value. Let's break this down into manageable subtasks:

1. Gather the initial value and final value for which you want to calculate percentage change.
2. Calculate the percentage change using the formula: 
   \( \text{Percentage Change} = \left( \frac{\text{Final Value} - \text{Initial Value}}{\text{Initial Value}} \right) \times 100 \)

I'll assign the tasks now:

1. web_search_agent : Find definitions and examples of percentage change.
2. data_manipulator_agent : Prepare to calculate percentage change once values are provided. 

Please provide the initial and final values whenever you're ready.
---------- WebSearchAgent ----------
[FunctionCall(id='call_3MmDWnA6quEJRopzwJq6Cenl', arguments='{"query":"definition and exampl

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the percentage change?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=127, completion_tokens=163), metadata={}, content="To provide you with a detailed response on percentage change, I first need to clarify your request. Percentage change usually involves comparing an initial value to a final value. Let's break this down into manageable subtasks:\n\n1. Gather the initial value and final value for which you want to calculate percentage change.\n2. Calculate the percentage change using the formula: \n   \\( \\text{Percentage Change} = \\left( \\frac{\\text{Final Value} - \\text{Initial Value}}{\\text{Initial Value}} \\right) \\times 100 \\)\n\nI'll assign the tasks now:\n\n1. web_search_agent : Find definitions and examples of percentage change.\n2. data_manipulator_agent : Prepare to calculate percentage change once values are provided. \n\